In [70]:
%matplotlib inline
import pandas as pd
import numpy as np 
from path import Path 
import matplotlib.pyplot as plt 
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import EasyEnsembleClassifier



In [44]:
#set path and load dataframe for input and target dfs
sml_df_file_path = Path('./analysis/sml_df.csv')
sml_df = pd.read_csv(sml_df_file_path)

target_df_file_path = Path('./analysis/target_df.csv')
target_df = pd.read_csv(target_df_file_path)

In [45]:
sml_df.head()

,action_taken,loan_amount_000s,msamd,applicant_income_000s,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,loan_type_name,property_type_name,loan_purpose_name,owner_occupancy_name,preapproval_name,applicant_ethnicity_name,co_applicant_ethnicity_name,purchaser_type_name
0,3,570.0,40900.0,144.0,4824.0,37.230000,75200.0,57.419998,818.0,1626.0,Conventional,One-to-four family dwelling (other than manufa...,Home improvement,Owner-occupied as a principal dwelling,Not applicable,Not Hispanic or Latino,Not Hispanic or Latino,Loan was not originated or was not sold in cal...
1,3,185.0,40140.0,51.0,7404.0,57.520000,63200.0,116.010002,1215.0,1743.0,Conventional,One-to-four family dwelling (other than manufa...,Home purchase,Not owner-occupied as a principal dwelling,Not applicable,Not Hispanic or Latino,No co-applicant,Loan was not originated or was not sold in cal...
2,1,1079.0,36084.0,278.0,3372.0,33.189999,97400.0,141.740005,592.0,1105.0,Conventional,One-to-four family dwelling (other than manufa...,Home purchase,Owner-occupied as a principal dwelling,Not applicable,"Information not provided by applicant in mail,...","Information not provided by applicant in mail,...",Loan was not originated or was not sold in cal...
3,3,417.0,36084.0,125.0,8787.0,65.129997,97400.0,97.269997,1463.0,2164.0,Conventional,One-to-four family dwelling (other than manufa...,Refinancing,Owner-occupied as a principal dwelling,Not applicable,Not Hispanic or Latino,Not Hispanic or Latino,Loan was not originated or was not sold in cal...
4,2,379.0,40900.0,86.0,5356.0,23.100000,75200.0,126.690002,1711.0,2102.0,FHA-insured,One-to-four family dwelling (other than manufa...,Refinancing,Owner-occupied as a principal dwelling,Not applicable,Not Hispanic or Latino,No co-applicant,Loan was not originated or was not sold in cal...


In [46]:
sml_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1477033 entries, 0 to 1477032
Data columns (total 18 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   action_taken                    1477033 non-null  int64  
 1   loan_amount_000s                1477033 non-null  float64
 2   msamd                           1477033 non-null  float64
 3   applicant_income_000s           1477033 non-null  float64
 4   population                      1477033 non-null  float64
 5   minority_population             1477033 non-null  float64
 6   hud_median_family_income        1477033 non-null  float64
 7   tract_to_msamd_income           1477033 non-null  float64
 8   number_of_owner_occupied_units  1477033 non-null  float64
 9   number_of_1_to_4_family_units   1477033 non-null  float64
 10  loan_type_name                  1477033 non-null  object 
 11  property_type_name              1477033 non-null  object 
 12  

In [47]:
# define clean target_df 
def action_taken_binary(member):
    if member == 1:
        return 0
    else:
        return 1

In [48]:
sml_df.action_taken = sml_df.action_taken.apply(action_taken_binary)

In [49]:
# Create features
X = sml_df.copy()
X = X.drop(columns='action_taken')
X = pd.get_dummies(X)

# Create target
y = sml_df[['action_taken']]


In [50]:
X.describe()

,loan_amount_000s,msamd,applicant_income_000s,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,loan_type_name_Conventional,...,purchaser_type_name_Affiliate institution,"purchaser_type_name_Commercial bank, savings bank or savings association",purchaser_type_name_Fannie Mae (FNMA),purchaser_type_name_Farmer Mac (FAMC),purchaser_type_name_Freddie Mac (FHLMC),purchaser_type_name_Ginnie Mae (GNMA),"purchaser_type_name_Life insurance company, credit union, mortgage bank, or finance company",purchaser_type_name_Loan was not originated or was not sold in calendar year covered by register,purchaser_type_name_Other type of purchaser,purchaser_type_name_Private securitization
count,1.477033e+06,1.477033e+06,1.477033e+06,1.477033e+06,1.477033e+06,1.477033e+06,1.477033e+06,1.477033e+06,1.477033e+06,1.477033e+06,...,1.477033e+06,1.477033e+06,1.477033e+06,1.477033e+06,1.477033e+06,1.477033e+06,1.477033e+06,1.477033e+06,1.477033e+06,1.477033e+06
mean,3.999415e+02,3.493538e+04,1.445669e+02,5.873621e+03,5.550147e+01,7.549268e+04,1.182309e+02,1.212891e+03,1.732202e+03,8.086509e-01,...,7.284874e-03,5.798313e-02,1.565686e-01,1.354066e-05,9.853402e-02,6.226130e-02,5.148971e-02,5.349054e-01,2.053509e-02,1.042428e-02
std,9.014832e+02,9.458675e+03,4.622419e+02,2.891405e+03,2.461725e+01,1.705402e+04,4.882625e+01,6.885472e+02,8.734476e+02,3.933634e-01,...,8.504005e-02,2.337116e-01,3.633937e-01,3.679740e-03,2.980354e-01,2.416296e-01,2.209945e-01,4.987803e-01,1.418218e-01,1.015658e-01
min,1.000000e+00,1.124400e+04,1.000000e+00,0.000000e+00,0.000000e+00,4.730000e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.190000e+02,3.108400e+04,6.800000e+01,4.130000e+03,3.490000e+01,6.320000e+04,8.307000e+01,7.520000e+02,1.162000e+03,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,3.360000e+02,4.014000e+04,1.030000e+02,5.360000e+03,5.406000e+01,7.390000e+04,1.112200e+02,1.097000e+03,1.576000e+03,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
75%,4.800000e+02,4.174000e+04,1.600000e+02,6.848000e+03,7.667000e+01,8.560000e+04,1.456700e+02,1.522000e+03,2.106000e+03,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
max,4.750000e+05,4.970000e+04,4.420000e+05,3.945400e+04,1.000000e+02,1.315000e+05,3.987000e+02,5.229000e+03,7.379000e+03,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [51]:
y.value_counts()

action_taken
0               810866
1               666167
dtype: int64

In [52]:

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
y_test.shape

(369259, 1)

In [53]:
# scale the model
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [59]:
# SMOTE oversampling
smt = SMOTE(random_state=1)
X_resampled, y_resampled = smt.fit_resample(X_train, y_train)
y_resampled = y_resampled.ravel()
Counter(y_resampled)


KeyboardInterrupt: 

In [61]:
y_resampled = y_resampled['action_taken'].ravel()

In [62]:
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)

In [64]:
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=['Actual 0','Actual 1'], columns=['Predicted 0','Predicted 1'])
cm_df.head()

,Predicted 0,Predicted 1
Actual 0,154006,48820
Actual 1,19274,147159


In [65]:
# Print the imbalanced classification report
print('Imbalanced Classification Report - SMOTE Oversampling')
print(classification_report_imbalanced(y_test, y_pred))

Imbalanced Classification Report - SMOTE Oversampling
                   pre       rec       spe        f1       geo       iba       sup

          0       0.89      0.76      0.88      0.82      0.82      0.66    202826
          1       0.75      0.88      0.76      0.81      0.82      0.68    166433

avg / total       0.83      0.82      0.83      0.82      0.82      0.67    369259



In [68]:
# EASY ENSEMBLE ADABOOST CLASSIFIER 
eec_model = EasyEnsembleClassifier()
y_train = y_train['action_taken'].ravel()
eec_model.fit(X_train, y_train)
y_pred = eec_model.predict(X_test)
Counter(y_pred)

Counter({1: 188000, 0: 181259})

In [71]:
# Calculated the balanced accuracy score
print('Balanced Accuracy Score - Easy Ensemble AdaBoost Classifier')
print(balanced_accuracy_score(y_test, y_pred))

Balanced Accuracy Score - Easy Ensemble AdaBoost Classifier
0.8236250364422771


In [72]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index=['Actual 0','Actual 1'], columns=['Predicted 0','Predicted 1'])
print('Confusion Matrix - Easy Ensemble AdaBoost Classifier')
cm_df.head()

Confusion Matrix - Easy Ensemble AdaBoost Classifier


,Predicted 0,Predicted 1
Actual 0,158732,44094
Actual 1,22527,143906


In [73]:
# Print the imbalanced classification report
print('Imbalanced Classification Report - Easy Ensemble AdaBoost Classifier')
print(classification_report_imbalanced(y_test, y_pred))

Imbalanced Classification Report - Easy Ensemble AdaBoost Classifier
                   pre       rec       spe        f1       geo       iba       sup

          0       0.88      0.78      0.86      0.83      0.82      0.67    202826
          1       0.77      0.86      0.78      0.81      0.82      0.68    166433

avg / total       0.83      0.82      0.83      0.82      0.82      0.68    369259

